<p style='text-align: center'>
    <img src='images/cesi.png' width="20%">
    <div style='text-align: center'>Rima Benrejeb, Thomas Mattone, Bastien Reynaud, Badreddine Ferragh</div>
</p>



# Leyenda - Autoencoder

## **Table Of Contents**
> 1. [Objective](#1)
> 2. [Data](#2)
> 3. [Notebook imports](#3)
> 4. [Data preparation](#4)
> 5. [Hyper-parameters](#5)
> 6. [Deep Neural Networks](#6)
> 7. [Convolutional Neural Networks](#7)
> 11. [Results](#8)

## Objective <a class="anchor" id="1"></a> 

In this notebook we will discuss the denoising of photos using algorithms relied on convolution auto-encoders in order to facilitate their processing.

Please note that I you run this notebook code on your own, **you may encounter different accuracy results** than the ones we obtained. <br>
This is due on the way the `model.evaluate()` function works in TensorFlow.

## Data <a class="anchor" id="2"></a> 

The dataset is composed of 150 .jpg images, comming in different dimensions.

Because of the low number of images, in order to obtain significants results, we will augment this dataset to increase the number of trainable items.

## Notebook imports <a class="anchor" id="3"></a> 

In [ ]:
%load_ext watermark

import warnings
warnings.filterwarnings("ignore")

# File manipulation
import os
import pathlib
import shutil
import wget
import zipfile

# Data manipulation
import numpy as np

# Machine Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Input, Flatten, ReLU, Reshape
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(1234)

# Image manipulation
import imgaug.augmenters as iaa
import imgaug as ia
import imghdr
from PIL import ImageFile

# Options for PIL
ImageFile.LOAD_TRUNCATED_IMAGES = True

ia.seed(42)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import visualkeras

# Options for seaborn
sns.set_style('darkgrid')
%matplotlib inline

# Utils
import leyenda_utils as lu
%watermark -p watermark,wget,numpy,sklearn,tensorflow,PIL,matplotlib,seaborn,visualkeras,imgaug

## Data preparation <a class="anchor" id="4"></a>

First we verify the validaity of the images from the dataset.

In [ ]:
DATA_PATH = 'data/image_noise'

data_dir = pathlib.Path(os.path.join(DATA_PATH))
invalid_images = []

for file in list(data_dir.glob('*/*.*')):
    if imghdr.what(file) not in ['jpeg', 'png']:
        invalid_images.append(file)
        
print(f'{len(invalid_images)} invalids images')

In the case we encounter no-usable images, we move them into a different directory.

In [ ]:
import glob
from skimage import io

images = []

for path in glob.glob(DATA_PATH + '/*'):
    images.append(io.imread(path))

Then we load the images, and preprocess them into a `IMG_H` by `IMG_W` size.

In [ ]:
import imgaug.augmenters as iaa
import imgaug as ia

IMG_H, IMG_W = 180, 180

preprocess = iaa.Sequential([
    iaa.Resize((IMG_H, IMG_W))
])

images_pre = np.array(preprocess(images=images))

print(images_pre.shape)

To augment our image base, we apply the following transformation on each basic images:
- an horizontal and vertical flip, each with 50% chance
- a zoom-in between 1 and 1.5
- a gamma and sigmoid contrast modification

This procedure is repeated as many time as the `AUGMENTED_FACTOR` value.

In [ ]:
AUGMENTATION_FACTOR = 10

seq = iaa.Sequential([
    iaa.HorizontalFlip(0.5),
    iaa.VerticalFlip(0.5),
    iaa.Affine(scale=(1, 1.5)),
    iaa.GammaContrast((0.5, 1),
                      per_channel=True),
    iaa.SigmoidContrast(gain=(3, 10),
                        cutoff=(0.4, 0.6),
                        per_channel=True)
])

images_aug = images_pre

for i in range(AUGMENTATION_FACTOR):
    images_aug = np.concatenate((images_aug, seq(images=images_pre)), axis=0)
    
print(images_aug.shape)

Next step is to split the augmented dataset into training, validation and testing set.

In [ ]:
fractions = np.array([0.8, 0.1, 0.1])

train_split, val_split, test_split = np.array_split(images_aug,
                                                    (fractions[:-1].cumsum() * len(images_aug)).astype(int))
                                                    
print(train_split.shape)
print(val_split.shape)
print(test_split.shape) 

Finally, we create a noised version of each set by randomly adding some gaussian and laplace noise.

In the same time, we perform feature scaling by mapping the images pixel values between 0 and 1.

In [ ]:
noise = iaa.Sequential([
    iaa.AdditiveGaussianNoise(scale=(0, 0.2 * 255),
                              per_channel=True),
    iaa.AdditiveLaplaceNoise(scale=(0, 0.2 * 255))
])

train = train_split / 255
val = val_split / 255
test = test_split / 255

train_noise = noise(images=train_split).astype(np.float32) / 255
val_noise = noise(images=val_split).astype(np.float32) / 255
test_noise = noise(images=test_split).astype(np.float32) / 255

In [ ]:
lu.compare_image_sets([train, train_noise], 10,
                      labels=['original', 'noise'],
                      size=(15, 4))

## Hyper-parameters <a class="anchor" id="5"></a> 

In order to be able to compare the model troughout the notebook, we chose to training them using the same configuration.

In [ ]:
BATCH_SIZE = 16
NUM_EPOCH = 1000
LOSS = MeanSquaredError()
OPTIMIZER = Adam(1e-3)
METRICS = [lu.SSIM]

To measure to accruacy during, we will use the `SSIM` metric.

SSIM is a value allowing the compare the similarity between two images:
- **1** indicates perfect similarity
- **0** indicates no similarity
- **-1** indicates perfect anti-correlation

## Autoencoder <a class="anchor" id="6"></a> 

This autoencoder architecture works on the basis of 2 concepts that will allow us to take a noisy image as input:
- **Encoder**: The purpose of this step is to downsample the input image. To do this, we will use convolution operations as seen previously in the first notebook. This will result in reducing the size of the input image. As a result of the convolution operations, we will obtain a vector representation of our image, called **latent space**.
- **Decoder**: The second part, called decoder, takes as input the latent space generated by the encoder. Its objective will be to reconstruct the image by removing the noise. To do this, we use layers of transposed convolutions.

<img src='images/autoencoder.png'>

## Deep Neural Network <a class="anchor" id="6"></a> 

We began with a dummy `dnn_1`, to be able to visualize how behave a poorly designed network.

It is composed of `Dense` layers activated with ReLU function.

In [ ]:
dnn_1 = Sequential([
    Flatten(input_shape=(IMG_H, IMG_W, 3)),
    ####
    Dense(units=128, activation='relu'),
    Dense(units=64, activation='relu'),
    ####
    Dense(units=32, activation='relu'),
    ####
    Dense(units=64, activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=IMG_H * IMG_W * 3, activation='relu'),
    ####
    Reshape(target_shape=(IMG_H, IMG_W, 3))
], name='dnn_1')

dnn_1.compile(loss=LOSS,
              optimizer=OPTIMIZER,
              metrics=METRICS)

visualkeras.layered_view(dnn_1, scale_xy=0.6)

In [ ]:
if lu.is_model_already_trained(dnn_1):
    lu.load_model_training(dnn_1)
else:
    dnn_1.fit(train_noise, train,
              batch_size=BATCH_SIZE,
              epochs= NUM_EPOCH,
              validation_data = (val_noise, val))
            
    lu.save_model_training(dnn_1)

Despite the `loss` nad `val_loss` going down quickly, the `accuracy` struggle to pass the 35% and the model is overfitting a bit with a `val_accuracy` over 25%.

In [ ]:
lu.plot_model_history(dnn_1)

`dnn_1` barely reach the 30% of accuracy.

In [ ]:
dnn_1.evaluate(test_noise, test)

If we have a lokk at the reconstructed image, we can see nevertheless that the model produce 'ghost' images with the global shape and colors of the original images.

In [ ]:
preds_dnn_1 = dnn_1.predict(test_noise)

lu.compare_image_sets([test, test_noise, preds_dnn_1], 10,
                      labels=['orignal', 'noise', 'dnn_1'],
                      size=(15, 6))

## Convolutional Neural Network <a class="anchor" id="7"></a> 

As already explained in the previous notebook, the reason of the bad performance of `dnn_1` is that Deep Neural Networks are not designed for image analysis.

Convolutional Neural Networks helps detecting image features such as edges, color gradient or other parameters making an image unique.

<img src='images/convolution.gif'>

The follwing model includes 4 `Conv2D` layers activated by a ReLU function and a latent space of 32.

In [ ]:
cnn_1 = Sequential([
    Input(shape=(IMG_H, IMG_W, 3)), 
    ###
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
    ###
    Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
    ###
    Conv2D(filters=3, kernel_size=(3, 3), padding='same', activation='sigmoid')
], name='cnn_1')

cnn_1.compile(loss=LOSS,
              optimizer=OPTIMIZER,
              metrics=METRICS)

visualkeras.layered_view(cnn_1, scale_xy=0.9, scale_z=0.1)

In [ ]:
if lu.is_model_already_trained(cnn_1):
    lu.load_model_training(cnn_1)
else:
    cnn_1.fit(train_noise, train,
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCH,
              validation_data = (val_noise, val))
            
    lu.save_model_training(cnn_1)

Adn we can already see a big improvement compare to `dnn_1`.

`cnn_1` follow the same behavior as its predecessor but the `loss` and `val_loss` decrease below 0.003. <br>
On the other side, the `accuracy` and `val_accuracy` are much closer and above the 75% mark.

In [ ]:
lu.plot_model_history(cnn_1)

In [ ]:
lu.load_model_training(dnn_1)

lu.plot_models_history([dnn_1, cnn_1])

On the testing set, `cnn_1` reaches 78% of accuracy. 

In [ ]:
cnn_1.evaluate(test_noise, test)

Now the reconstructions are much closer to the originals. <br>
However, we can still notice some image artefacts due to the amount of noise that was applied at the beggining.

In [ ]:
preds_cnn_1 = cnn_1.predict(test_noise)

lu.compare_image_sets([test, test_noise, preds_cnn_1], 10,
                      labels=['orignal', 'noise', 'cnn_1'],
                      size=(15, 6))

## Results <a class="anchor" id="8"></a>  

In [ ]:
lu.compare_image_sets([test, test_noise, preds_dnn_1, preds_cnn_1], 10,
                      labels=['original', 'noise', 'dnn_1', 'cnn_1'],
                      size=(15, 8))